In [1]:
# words followed by 'client:'
def extract_organisation(text):
    lines = text.lower().split('\n')
    orgs = list()
    for line in lines:
        if line.find('client:')>-1:
            if len(line.split(':'))>1:
                ent = line.split(':')[1].split(',')[0].strip()
                if ent != '':
                    org = nltk.tokenize.sent_tokenize(ent)[0]
                    #print(org)
                    orgs.append(org)
    #print(orgs)
    return orgs

In [16]:
import re
def clean_text(text):
    text = text.strip()
    text = re.sub(r'\t+',' ',text)
    text = re.sub(r'\n+','\n',text)
    return text

In [17]:
print('\n\nab\t\tcd')



ab		cd


In [18]:
clean_text('\n\nab\t\tc\n\nd')

'ab c\nd'

In [3]:
#words followed by environment:
def extract_skills(text):
    skills = list()
    for line in text.split('\n'):
        line = line.lower()
        if line.find('environment:')>-1:
            skills.extend(line.replace('environment:','').split(','))
    return skills

In [4]:
#mostly first line would be name if not enter manually
def extract_Name(text):
    name = nltk.tokenize.sent_tokenize(text.strip())[0].split('\n')[0]
    print('extracted name',name)
    res = input()
    if res == 'y':
        names.append(name)
    elif res == 'e':
        print(text.strip().split('\n')[:5])
        names.append(input())
    else:
        names.append(input())

In [ ]:
def extract_exp(text):
    doc = nlp(text)
    entities = dict()
    for _ in doc.ents:
        label = _.label_.lower()
        entity = _.text.lower()
        if label not in entities:
            entities[label] = list()
            entities[label].append(entity)
        else:
            entities[label].append(entity)
    print('experience',entities['date'][0])
    res = input()
    if res == 'y':
        return entities['date'][0]
    elif res == 'e':
        print(text.strip().split('\n')[:10])
        return input()
    else:
        return input()

In [ ]:
#!/usr/bin/env python

import tika
tika.initVM()
from tika import parser
def read_file():
    parsed = parser.from_file('./Resumes/Venkat_BA.docx')
    print(parsed["metadata"])
    print(parsed["content"])

In [5]:
# Dowloaded linkedin skills from kaggle and used it search in our resume, used words to filter unwanted skills

In [6]:
def filter_linkedin_skills():
    all_skills =list()
    with open('linkedin skill') as fp:
        for line in fp.readlines():
            all_skills.append(line.strip())
    
    # get all the words in all files
    vocab = set()
    for file in tqdm.tqdm(files):
        text = read_file(file)
        sents = nltk.tokenize.sent_tokenize(text)
        for sent in sents:
            for word in nltk.tokenize.word_tokenize(sent):
                vocab.add(word)
    
    all_skills = vocab.intersection(all_skills)
    return all_skills

In [7]:
def get_matches():
    all_file_matches = list()
    for file in tqdm.tqdm(files):
        text = read_file(file)
        all_matches = dict()
        for skill in all_skills:
            if skill == '':
                continue
            matches = [_.span() for _ in re.finditer(r'\b(%s)\b' % re.escape(skill), text)]
            if len(matches)>0:
                all_matches[skill] = matches
        all_file_matches.append(all_matches)
    return all_file_matches

In [8]:
def get_entity_indexes(doc):
    doc = resume_entities.loc[ix]
    text = read_file(doc['file']).lower()
    matches = list()
    print(i,doc['file'])
    for col in ['skill_matches','org_matches','experience','person name','city']:
        entities = doc[col]
        if col == 'skill_matches':
            col = 'skill'
            entities = literal_eval(entities)
        elif col=='org_matches':
            col = 'org'
            entities = literal_eval(entities)
        elif col == 'person name':
            col = 'person'
        if entities == 'None':
            continue
        if type(entities) in [dict,list]:
            for ent in entities:
                matches.extend([_.span()+(col,) for _ in re.finditer(r'\b(%s)\b' % re.escape(ent.lower().strip()), text)])
        elif type(entities) is str:
            temp = [_.span()+(col,) for _ in re.finditer(r'\b(%s)\b' % re.escape(entities.lower().strip()), text)]
            if len(temp)>0:
                matches.append(temp[0])
        else:
            continue
    return matches